In [1]:
import sys
sys.path.append("..")

In [2]:
file = '/Users/knpob/Territory/Kolmo/code/foot-measure/data/stl/15032021_001_l 1.stl'
label_path = '/Users/knpob/Territory/Kolmo/code/foot-measure/data/stl/15032021_001_l 1.pkl'
is_plot = True

## Load landmarks labelling

In [3]:
import pandas as pd

df = pd.read_pickle(label_path)
df

coord                                                                 x  \
file                                               landmark               
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1         56.321947   
                                                   P10      -182.886558   
                                                   P11      -123.357072   
                                                   P12      -125.791490   
                                                   P2         42.513113   
                                                   P3          8.947728   
                                                   P4         -2.691079   
                                                   P5        -25.348262   
                                                   P6        -50.272838   
                                                   P7        -68.890782   
                                                   P8       -154.958698   
                                                   P9       -153.365039   

coord                                                                 y  \
file                                               landmark               
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1         68.714972   
                                                   P10        62.749513   
                                                   P11        97.606296   
                                                   P12        28.127211   
                                                   P2        111.336435   
                                                   P3         13.609714   
                                                   P4        114.532845   
                                                   P5         10.151830   
                                                   P6         68.595718   
                                                   P7         72.850203   
                                                   P8         88.408319   
                                                   P9         24.436464   

coord                                                                z  
file                                               landmark             
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        -6.487760  
                                                   P10      -17.147384  
                                                   P11      -59.674913  
                                                   P12      -61.804787  
                                                   P2        -6.030457  
                                                   P3        -8.369461  
                                                   P4       -18.445211  
                                                   P5       -11.298273  
                                                   P6       -52.907105  
                                                   P7       -63.107783  
                                                   P8       -12.479075  
                                                   P9       -10.606368

## Local frame

In [4]:
import pyvista as pv
from measure import frame
from mesh4d.analyse import crave

mesh = crave.fix_pvmesh_disconnect(pv.read(file))

# foot bottom cropped for estimating foot local frame
mesh_clip = frame.foot_clip(mesh, df, file, ['P2', 'P3', 'P4', 'P5', 'P8', 'P9'], invert=False)

if is_plot:
    mesh_clip.plot()

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca78d2da60_0&reconnect=auto' style='widt…

In [5]:
from measure import visual

axes_frame, origin = frame.estimate_foot_frame(mesh, file, df, invert=False)

if is_plot:
    visual.plot_axes(origin, axes_frame, mesh)

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca98d2ac10_1&reconnect=auto' style='widt…

In [7]:
mesh_clip = frame.foot_clip(mesh, df, file, invert=False)
mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)

if is_plot:
    mesh_local.plot()

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca8c27f940_3&reconnect=auto' style='widt…

In [8]:
df_local = frame.df2local(df, axes_frame, origin)
df_local

coord                                                                 x  \
file                                               landmark               
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        117.514572   
                                                   P10      -118.677353   
                                                   P11       -53.881743   
                                                   P12       -69.194022   
                                                   P2        111.868776   
                                                   P3         60.717836   
                                                   P4         67.995581   
                                                   P5         26.364959   
                                                   P6         12.565366   
                                                   P7         -4.979226   
                                                   P8        -86.448825   
                                                   P9        -96.763419   

coord                                                                y  \
file                                               landmark              
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1        23.480702   
                                                   P10      -15.304367   
                                                   P11      -39.375917   
                                                   P12       28.382871   
                                                   P2       -20.944913   
                                                   P3        68.778177   
                                                   P4       -32.738062   
                                                   P5        65.745090   
                                                   P6         2.836922   
                                                   P7        -5.012254   
                                                   P8       -35.227593   
                                                   P9        27.952251   

coord                                                                z  
file                                               landmark             
/Users/knpob/Territory/Kolmo/code/foot-measure/... P1         7.482238  
                                                   P10       16.371141  
                                                   P11       58.672851  
                                                   P12       62.149512  
                                                   P2         6.077941  
                                                   P3        10.073834  
                                                   P4        18.070705  
                                                   P5        12.799645  
                                                   P6        53.053233  
                                                   P7        63.021184  
                                                   P8        11.419395  
                                                   P9        10.817816

## Metrics

In [9]:
results = [
    {
        'file': 'description',
        'FL': 'foot length (mm)',
        'MBL': 'medial ball length (mm)',
        'LBL': 'lateral ball length (mm)',
        'ABW': 'anatomical ball width (mm)',
        'OBW': 'orthogonal ball width (mm)',
        'OHW': 'orthogonal heel width (mm)',
        'BH': 'ball heigh (mm)',
        'IH': 'instep height (mm)',
        'BA': 'ball angle (°)',
        'T1A': 'toe 1 angle (°)',
        'T5A': 'toe 5 angle (°)',
        'ABG': 'anatomical ball girth (mm)',
        'IG': 'instep girth (mm)',
    }
]

In [10]:
from measure import metric

results.append(
    {
        'file': file,
        'FL': metric.fl(df_local, file),
        'MBL': metric.mbl(df_local, file),
        'LBL': metric.lbl(df_local, file),
        'ABW': metric.abw(df_local, file),
        'OBW': metric.obw(df_local, file),
        'OHW': metric.ohw(df_local, file),
        'BH': metric.bh(df_local, file),
        'IH': metric.ih(df_local, file),
        'BA': metric.ba(df_local, file),
        'T1A': metric.t1a(df_local, file),
        'T5A': metric.t5a(df_local, file),
        'ABG': metric.abg(df_local, file, mesh_local),
        'IG': metric.ig(df_local, file, mesh_local),
    }
)

In [11]:
df_metric = pd.DataFrame(results).set_index('file')
df_metric

,FL,MBL,LBL,ABW,OBW,OHW,BH,IH,BA,T1A,T5A,ABG,IG
file,,,,,,,,,,,,,
description,foot length (mm),medial ball length (mm),lateral ball length (mm),anatomical ball width (mm),orthogonal ball width (mm),orthogonal heel width (mm),ball heigh (mm),instep height (mm),ball angle (°),toe 1 angle (°),toe 5 angle (°),anatomical ball girth (mm),instep girth (mm)
/Users/knpob/Territory/Kolmo/code/foot-measure/data/stl/15032021_001_l 1.stl,236.191925,186.672934,145.042312,107.050567,101.516239,63.179844,53.053233,63.021184,67.927367,22.209286,13.165781,250.412361,245.483957


### Visualization

In [12]:
if is_plot:
    settings = {
        'FL': ['P1', 'P10', 'x'],
        'MBL': ['P4', 'P10', 'x'],
        'LBL': ['P5', 'P10', 'x'],
        'OBW': ['P4', 'P3', 'y'],
        'OHW': ['P9', 'P8', 'y'],
    }

    for name, [landmark1, landmark2, axis] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_dist_along_axis(scene, df_local, file, landmark1, landmark2, axis, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca99ba2940_4&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca8c274610_5&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca8c48c760_6&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca8c426580_7&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca99c46dc0_8&reconnect=auto' style='widt…

In [13]:
if is_plot:
    settings = {
        'BH': 'P6',
        'IH': 'P7',
    }

    for name, landmark in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_height(scene, df_local, file, landmark, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca842f6fa0_9&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca99c88d00_10&reconnect=auto' style='wid…

In [14]:
if is_plot:
    settings = {
        'BA': ['P4', 'P5', 'P8'],
        'T1A': ['P4', 'P2', 'P8'],
        'T5A': ['P5', 'P3', 'P9'],
    }

    for name, [landmark_origin, landmark1, landmark2] in settings.items():
        scene = pv.Plotter()
        scene.add_mesh(mesh_local, opacity=0.1)
        visual.plot_angle(scene, df_local, file, landmark_origin, landmark1, landmark2, actue_angle=True, name=name)
        scene.show()

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca8ca82910_11&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca842f6d00_12&reconnect=auto' style='wid…

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca99c88550_13&reconnect=auto' style='wid…

In [15]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_2landmarks(scene, df_local, file, mesh_local, ['P4', 'P5'], tangent_axis='z', name='ABG')
    scene.show()

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca99ba29a0_14&reconnect=auto' style='wid…

In [16]:
if is_plot:
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_circ_pass_landmark(scene, df_local, file, mesh_local, 'P6', 'x', name='IG')
    scene.show()

Widget(value="<iframe src='http://localhost:54525/index.html?ui=P_0x7fca83ea7850_15&reconnect=auto' style='wid…